"""
Created on Wed Nov  6 14:52:11 2019
@author: Jamie Billington
"""
Notebook outlining an approach to build a classifier using keras neural networks to discriminate between 
insulators at chromatin transition regions and non insulating sequences. Prospective insulators taken from 'Genome-wide prediction and analysis of human chromatin boundary elements' Wang et al. 2012 


In [12]:
##############################################################################
## Initialise
##############################################################################
# Import libraries

import pandas as pd
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import random

import keras
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.convolutional import Conv1D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import CSVLogger
import keras.backend as K


import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import scipy.sparse as sparse
import re

First we read in the DNA sequences for our regions of interest- chromatin transition regions (CTR) and either a)random sequences or b) sequences flanking either side of the CTR

In [13]:
os.chdir('/Users/jamie/Documents/Coding/Machine Learning/')

#Import 8kb regions generated in R which contain either a random sample of the genome or a chromatin transition region
random_8kb_sequences = pd.read_csv('1000_8kb_random_samples_upper.tab',sep='\t',names=['Region','Sequence'])
sequence_extract=pd.Series(random_8kb_sequences['Sequence'])
sliced=sequence_extract.str.slice(start=3500,stop=4500)
random_sequences=pd.DataFrame(sliced)
random_sequences=random_sequences[0:2761]

# Option to Slice out the 1000bp predicted inuslator chunk at the core of each Chromatin transition region
ctr_sequences = pd.read_csv('insulator_library.tab',sep='\t',names=['Region','Sequence'])
sequence_extract=pd.Series(ctr_sequences['Sequence'])
sliced=sequence_extract.str.slice(start=3500,stop=4500)
insulator_sequences=pd.DataFrame(sliced)



combined_sequences=insulator_sequences.append(random_sequences)
print(combined_sequences.head())

insulator_dataset= combined_sequences.copy()
## Running optimised model on whole dataset

                                            Sequence
0  TTATCTGAAAAATCACAGCTTGGGTGCATGTGAAGCCAGAGGAGCA...
1  AGGGCCAGTGTCAGAGCTCCCCTGTGCCCCTGTCCCGCCACAGCCA...
2  GTTAACTAAAAAATGCAAAAAAATTAACAGCAGATTAAAGTCAGTG...
3  GAGATGCTGGTACACAGTTACTAGGGTATCTTCCCAAAATAAAACC...
4  GGGGGAAGAAGGTAGATGAGAACTCTGCAGTTTCTGCTTAGTTTTT...


Next one hot encode each of the insulator and control sequences in the library to generate a 4 x Number of base pairs array

Check that this works sequence entry 2 base 2 should be G [0,0,1,0]

In [16]:
# One hot encoder

def one_hot_encode(df, col, seq_len):
    # Dictionary returning one-hot encoding of nucleotides. 
    nuc_d = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0]}
    
    # Creat empty matrix.
    vectors=np.empty([len(df),seq_len,4])
    
    # Iterate through sequences and one-hot encode
    for i,seq in enumerate(df[col].str[:seq_len]): 
        seq = seq.lower()
        a = np.array([nuc_d[x] for x in seq])
        vectors[i] = a
    return vectors

one_hot_encoded=one_hot_encode(insulator_dataset,col='Sequence',seq_len=999)
print(one_hot_encoded[1][1]) 

[0. 0. 1. 0.]


Next comes labeling of the data for the basis of classification. Insulators are labelled as 1 (TRUE), non insulators assisgned 0 (FALSE)
Array of scores are encoded in y and the one hot coded array is relablled x for convential layout

In [20]:
##############################################################################
# Generating the output labels

ins=[1]*len(insulator_sequences)
data=pd.DataFrame(ins)
rando=[0]*len(random_sequences)
scores=data.append(rando)
y = np.ravel(scores)

########################

x=one_hot_encoded

from keras.utils import to_categorical
y_binary = to_categorical(y)


train_x,test_x,train_y,test_y=train_test_split(x,y, random_state=1)


array([1, 1, 1, ..., 0, 0, 0])

In [21]:
x

array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]],

       ...,

       [[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0.

In [22]:
y

array([1, 1, 1, ..., 0, 0, 0])

Code for building neural network model. Only a preliminary run to check the code works- no hyperparameter optimisation here. 

In [ ]:

################################################################################################################

def train_model(x, y, nb_epoch, conv_layers=2, inp_len=999, border_mode='same', nbr_filters=128, filter_len=8, 
                conv_dropout2=0.4, conv_dropout3=0, dense_layers=1, nodes1=100, nodes2=40, nodes3=70, dense_dropout1=0, 
                dense_dropout2=0, dense_dropout3=0, loss='binary_crossentropy'):
     
    model = Sequential()
    
    # Convolutional layers
    if conv_layers >= 1:
        model.add(Conv1D(activation="relu", input_shape=(inp_len, 4), padding=border_mode, filters=nbr_filters, 
                         kernel_size=filter_len))
        
    if conv_layers >= 2:
        model.add(Conv1D(activation="relu", input_shape=(inp_len, 1), padding=border_mode, filters=nbr_filters, 
                         kernel_size=filter_len))
        model.add(Dropout(conv_dropout2))
        
    if conv_layers >= 3:
        model.add(Conv1D(activation="relu", input_shape=(inp_len, 1), padding=border_mode, filters=nbr_filters, 
                         kernel_size=filter_len))
        model.add(Dropout(conv_dropout3))
    
    # Flatten
    model.add(Flatten())

    # Dense layers
    if dense_layers >= 1:
        model.add(Dense(nodes1, activation='relu'))
        model.add(Dropout(dense_dropout1))
        
    if dense_layers >= 2:
        model.add(Dense(nodes2, activation='relu'))
        model.add(Dropout(dense_dropout2))
        
    if dense_layers >= 3:
        model.add(Dense(nodes3, activation='relu'))
        model.add(Dropout(dense_dropout3))
        
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    
    model.fit(x, y, batch_size=10, epochs=nb_epoch, verbose=1)
    return model

# Run model
model = train_model(train_x,train_y, nb_epoch=10)
predictions = model.predict(test_x)
######################################################################
#model.save('insulator_training_parameters.h5')
#######################################################################

Epoch 1/10
4141/4141 [==============================] - 244s 59ms/step - loss: 0.7081 - acc: 0.4943
Epoch 2/10
4141/4141 [==============================] - 257s 62ms/step - loss: 0.6935 - acc: 0.4907
Epoch 3/10
4141/4141 [==============================] - 225s 54ms/step - loss: 0.6932 - acc: 0.4832
Epoch 4/10
4141/4141 [==============================] - 226s 54ms/step - loss: 0.6933 - acc: 0.4979
Epoch 5/10
4141/4141 [==============================] - 630s 152ms/step - loss: 0.6933 - acc: 0.4938
Epoch 6/10
4141/4141 [==============================] - 214s 52ms/step - loss: 0.6933 - acc: 0.4890
Epoch 7/10
4141/4141 [==============================] - 219s 53ms/step - loss: 0.6933 - acc: 0.4977
Epoch 8/10
4141/4141 [==============================] - 221s 53ms/step - loss: 0.6933 - acc: 0.4977
Epoch 9/10
4141/4141 [==============================] - 516s 125ms/step - loss: 0.6933 - acc: 0.4943
Epoch 10/10
4141/4141 [==============================] - 219s 53ms/step - loss: 0.6933 - acc: 0.49

Evaluate the models precitions: 
Print confusion matrix for IDing false + and - (In basic state model does no better than random )
Pretty sure that given this data will get a lot of overfitting -samples number ~ 5000 length of sample 1000bp

In [12]:
#############################################################################################################################
predict_bin= np.where(predictions>=0.5, 1, 0)

# Analysis
cm_all = confusion_matrix(test_y, predict_bin)
print(cm_all)


test_acc = (cm_all[0,0]+cm_all[1,1])/np.sum(cm_all) # accuracy achieved on test data; number predicted correctly / total
pred_one_acc = cm_all[1,1]/(cm_all[0,1]+cm_all[1,1]) # proportion of ones predicted correctly on test data
pred_zero_acc = cm_all[0,0]/(cm_all[0,0]+cm_all[1,0]) # prcmoportion of zeros predicted correctly on test data
f0_5 = fbeta_score(predict_bin,test_y, beta=0.5)  # punishes false positives more (i.e. promoters that are predicted as being stress activated but aren't)


print("Test accuracy: " + str(round(test_acc,4)))
print("Ones accuracy: " + str(round(pred_one_acc,4)))  
print("Zeros accuracy: " + str(round(pred_zero_acc,4)))  
print("F0.5 score: " + str(round(f0_5,4)))



NameError: name 'predictions' is not defined